In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

# BARTpho-syllable
# syllable_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
# bartpho_syllable = AutoModel.from_pretrained("vinai/bartpho-syllable")
# TXT = "Chúng tôi là những nghiên cứu viên."
# input_ids = syllable_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_syllable(input_ids)

# BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word-base")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word-base")
# to cuda
bartpho_word.to("cuda")
bartpho_word.eval()
for param in bartpho_word.parameters():
    param.requires_grad = False
# TXT = "Chúng_tôi là những nghiên_cứu_viên ."
# input_ids = word_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_word(input_ids.to("cuda"))

# input_ids = word_tokenizer.encode(TXT, return_tensors="pt").to("cuda")
# print(input_ids)
# # check if input_ids is in cuda
# features = bartpho_word(input_ids).last_hidden_state[0]

d:\AIChallenge\venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
# Word segmentation
import py_vncorenlp

# py_vncorenlp.download_model(save_dir=r"D:\VQA\Notebook\VnCoreNLP")

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"],
    save_dir=r"D:\AIChallenge\VnCoreNLP",
)

# change dir to C:\Users\Asus.LAPTOP-8EU9PHJL.000\Desktop\VQA\Notebook
import os

os.chdir(r"D:\AIChallenge")


def Word_Segmentation(text):
    output = rdrsegmenter.word_segment(text)
    if len(output) > 0:
        # join all words in the list
        res = ""
        for i in output:
            res += i + " "
        return res
    else:
        output = ""
    return output


print(Word_Segmentation("Chúng tôi là những nghiên cứu viên."))

Chúng_tôi là những nghiên_cứu_viên . 


In [3]:
import os


def scan_file_path_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            lst.append(os.path.join(root, file))
    return lst


def scan_file_name_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            # remove the extension
            file_name = os.path.splitext(file)[0]
            lst.append(file_name)
    return lst


audio_text = r"Data\rawData\Audio_text"

In [4]:
path_lst = scan_file_path_in_folder(audio_text)
name_lst = scan_file_name_in_folder(audio_text)

In [5]:
from tqdm import tqdm

for path, name in tqdm(zip(path_lst, name_lst)):
    # read text
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    # join text
    text = Word_Segmentation(text)
    # remove every '.', ',', '!', '?'
    text = text.replace(".", " ").replace(",", " ").replace("!", " ").replace("?", " ")
    # clean ' 'ArithmeticError
    text = " ".join(text.split())
    # write text
    with open(
        f"Data\PreporcessData\Audio_text_wseg\{name}.txt", "w", encoding="utf-8"
    ) as f:
        # write to each line
        f.write(text)
    # split the text into multiple parts with at most 100 words
    text = text.split(" ")
    lst_text = []
    for i in range(0, len(text), 100):
        lst_text.append([" ".join(text[i : i + 100])])
    lst_ids = [
        word_tokenizer.encode(
            i[0],
            return_tensors="pt",
            padding="max_length",
            max_length=128,
            truncation=True,
        ).to("cuda")
        for i in lst_text
    ]
    with torch.no_grad():
        # check if input_ids is in cuda
        lst_outputs = [bartpho_word(i) for i in lst_ids]
        lst_features = [i.last_hidden_state[0] for i in lst_outputs]
        # stack all features to a tensor
        features = torch.stack(lst_features)
    # save features
    torch.save(features, f"Data\PreporcessData\Audio_text_bartpho\{name}.pt")

363it [01:47,  3.39it/s]
